<a href="https://colab.research.google.com/github/mohantyshradhasai/demo/blob/main/multimodalchatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai --quiet
!pip install langchain --quiet
!pip install cohere --quiet
!pip install tiktoken --quiet
!pip install langchain_community --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.4/295.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")
os.environ['COHERE_API_KEY'] = userdata.get("COHERE_API_KEY")

In [ ]:
from langchain import Cohere, ConversationChain
from langchain.chat_models import ChatCohere
from langchain.memory import ConversationBufferMemory


In [ ]:
user_input=input("Your message:")
llm= Cohere(temperature=0, max_tokens=512)
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm,
                                 memory=memory,
                                 verbose=False
                                 )
while user_input!="quit":
    print("AI message ==>", conversation.predict(input=user_input))
    user_input=input("Your message: ")
while user_input =="quit":
    memory.clear()
    break

Your message:hi


/tmp/ipython-input-439229803.py:4: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(llm=llm,


AI message ==>  Hello! How can I help you today? 

Please feel free to ask me anything you like, and I will do my best to respond with accurate and useful information. If there's something specific you want to know, go ahead and ask, and I'll do my best to provide a detailed and nuanced response drawing on the wealth of knowledge I have access to. 

I'm here to have a conversation with you and to provide information on a wide range of topics. So, feel free to ask me anything, and I will do my best to make our conversation informative, engaging, and collaborative. 
Your message: quit


In [ ]:
#import pypdf
!pip install pypdf --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 7.3 MB/s eta 0:00:00


In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
import glob

In [ ]:
#write a code to load document
from google.colab import files

uploaded = files.upload()


Saving NCDGWR2023.pdf to NCDGWR2023.pdf


In [ ]:
pdf_files =list(uploaded.keys())
print("Uploaded PDFs:", pdf_files)

Uploaded PDFs: ['NCDGWR2023.pdf']


In [ ]:
from langchain.document_loaders import PyPDFLoader

full_text = ""
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pages = loader.load()
    print(pdf_file, len(pages))
    for page in pages:
        full_text += page.page_content

print(full_text[:1000])
print("Lines:", len(full_text.split("\n")))
print("Words:", len(full_text.split(" ")))
print("Characters:", len(full_text))


NCDGWR2023.pdf 460
National Compilation on 
DYNAMIC GROUND WATER RESOURCES OF INDIA, 2023 
 
 
 
 
 
 
 
 
 
  
Central Ground Water Board 
Department of Water Resources, 
River Development & Ganga Rejuvenation 
Ministry of Jal Shakti 
Government of India 
 
 
 
 
 
 
 
 
 
 
 
Faridabad 
September, 20236. q{. orftdr{fiq
vEsel€ruo
T. 5. Anitha Shyam
Member (South)
rfliftt $Iul,it
wrrfihdrrcrq
qo dsrtrq, rfl ftors oil{ riql {rdul frqrq
irfic{ftvctrS
Government of lndia
1{inistry of JalShrkti
oepartmant of Water Re3ourcer,
River oevelopment and G.nga Reruvon.tion
Cenkal Ground Water Board
Trov
3IlflIila -3qF rFrisa
Preface
Groundwater is a crucial resource, meeting to the water needs of agriculture, households, and industries in
our nation. Unfortunately, its development occurs haphazardly through individual enkepreneurs in an
unscientiflc manner, leading to over-exploitation in certain regions. To guarantee the sustainability of this
vital resource, prudent management is essential. Maki

splitting into chunks


In [ ]:
#splitting the text
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = text_splitter.split_documents(pages)

print(len(chunks))
print(chunks[0].page_content)  # access text


In [ ]:
!pip install ChromaDB --quiet

In [ ]:
#creating embeddings
from langchain_community.embeddings import CohereEmbeddings
import os
os.environ["COHERE_API_KEY"] = "bc9dj6eNiAgkZ614WHn81q3yGQeeBdmjh7jUEd6N"
embeddings = CohereEmbeddings(
    cohere_api_key=os.environ["COHERE_API_KEY"],
    model="embed-english-v3.0",
    user_agent="langchain"
)
from langchain_community.vectorstores import Chroma

arr_db = Chroma.from_documents(chunks, embeddings, persist_directory="arr_db")
arr_db.persist()




In [ ]:
retriever = arr_db.as_retriever()
result=retriever.get_relevant_documents(query="What is an array?Explain in 5 lines.")
for i in range(len(result)):
  print(result[i].page_content)

In [ ]:
#conversation and retreival chain
# Cohere LLM
llm = ChatCohere(cohere_api_key=os.environ["COHERE_API_KEY"], model="command-r")

# Conversational chain
conversational_rag = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=arr_db.as_retriever(),
    return_source_documents=True
)


In [ ]:
#import prompttemplate
from langchain.prompts import PromptTemplate
prompt_template = """Text: {context}
    Question: {question}
    you are a chatbot designed to assist the users.
    Answer only the questions based on the text provided. If the text doesn't contain the answer,
    reply that the answer is not available.
    keep the answers precise to the question"""

PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )
chain_type_kwargs = { "prompt" : PROMPT }

In [ ]:
pip install --upgrade langchain langchain-cohere cohere

In [ ]:
llm=Cohere(model="command", temperature=0)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
        llm = llm,
        retriever= arr_db.as_retriever(),
        memory = memory,
        combine_docs_chain_kwargs=chain_type_kwargs
)

In [ ]:
user_input=input("Your message:")
while user_input!="quit":
    response=conversation_chain({"question": user_input})
    print("AI message ==>", response["answer"])
    user_input=input("Your message: ")

In [ ]:
!pip install langchain cohere chromadb pytesseract pillow
!sudo apt-get install tesseract-ocr -y


In [ ]:
import os
from PIL import Image
import pytesseract
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import CohereEmbeddings
from langchain.vectorstores import Chroma


In [ ]:
#upload
from google.colab import files
uploaded = files.upload()
imagepath=list(uploaded.keys())[0]
image=Image.open(imagepath)

In [ ]:
#extract the image to string
text = pytesseract.image_to_string(image)
print(text)

In [ ]:
#splitting into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = text_splitter.split_text(text)
print(len(chunks))
print(chunks[0])

In [ ]:
#creating vector database
embeddings = CohereEmbeddings(
    cohere_api_key=os.environ["COHERE_API_KEY"],
    model="embed-english-v3.0",
    user_agent="langchain"
)
img_db=Chroma.from_texts(chunks, embeddings, persist_directory="img_db")
img_db.persist()

In [ ]:
retriever = img_db.as_retriever()
result=retriever.get_relevant_documents(query="What is the dress?")
for i in range(len(result)):
  print(result[i].page_content)

In [ ]:
llm=Cohere(model="command", temperature=0)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
        llm = llm,
        retriever= img_db.as_retriever(),
        memory = memory,
        combine_docs_chain_kwargs=chain_type_kwargs
)

In [ ]:
user_input=input("Your message:")
while user_input!="quit":
    response=conversation_chain({"question": user_input})
    print("AI message ==>", response["answer"])
    user_input=input("Your message: ")

In [ ]:
!pip install diffusers transformers accelerate safetensors


In [ ]:
from diffusers import StableDiffusionPipeline
import torch
#loading pipeline
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
#generate image
pipe=pipe.to("cuda")
image = pipe("an apple").images[0]
image.save("apple.png")

In [ ]:
display(image)

In [ ]:
!sudo apt-get update && sudo apt-get install espeak -y

In [ ]:
#load SWivid/F5-TTS
!pip install F5-TTS
#uninstall F5-TTS
!pip uninstall F5-TTS

In [ ]:
!pip install cohere gtts

In [ ]:
import cohere
from gtts import gTTS
from IPython.display import Audio
#setup cohere
co=cohere.Client('bc9dj6eNiAgkZ614WHn81q3yGQeeBdmjh7jUEd6N')
#generate text using cohere
documents = [
    {"title": "OS", "text": "Deadlock is a situation in operating systems where processes wait indefinitely."},
    {"title": "DBMS", "text": "Normalization reduces redundancy in databases."}
]
query="explain about deadlock situation."
response = co.rerank(
    query=query,
    documents=[d["text"] for d in documents],
    top_n=1
)
context = response.results[0].document
completion = co.generate(
    model="command-r-plus",
    prompt=f"Answer the question using this context: {context}\n\nQuestion: {query}\nAnswer:"
)

answer = completion.generations[0].text.strip()
print("Answer:", answer)
#convert to speech
tts = gTTS(answer)
tts.save("answer.mp3")
#play the audio
Audio("answer.mp3")

In [ ]:
!pip install SpeechRecognition pydub

import speech_recognition as sr
from pydub import AudioSegment
from google.colab import files

# Upload file
uploaded = files.upload()
filename = list(uploaded.keys())[0]   # get the uploaded filename

# Convert MP3 -> WAV (mono, 16kHz)
sound = AudioSegment.from_file(filename, format="mp3")
sound = sound.set_channels(1)       # mono
sound = sound.set_frame_rate(16000) # 16 kHz
wav_filename = "converted.wav"
sound.export(wav_filename, format="wav")

# Initialize recognizer
recognizer = sr.Recognizer()

# Process audio
with sr.AudioFile(wav_filename) as source:
    audio = recognizer.record(source)   # use record() not listen()

# Convert to text
try:
    text = recognizer.recognize_google(audio, language="en-US")
    print("You said:", text)
except sr.UnknownValueError:
    print("Google Speech Recognition could not understand audio")
except sr.RequestError as e:
    print(f"Could not request results; {e}")


In [ ]:

# =========================
# 1) Install dependencies
# =========================
!pip install -U gradio langchain langchain-community langchain-cohere cohere chromadb pypdf pillow pytesseract gtts SpeechRecognition pydub
!pip install -U diffusers transformers accelerate safetensors
!sudo apt-get install -y tesseract-ocr

# =========================
# 2) Imports & setup
# =========================
import os
import gradio as gr
import pytesseract
from PIL import Image
from pydub import AudioSegment
import speech_recognition as sr
import torch
import cohere
from gtts import gTTS

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import CohereEmbeddings
from langchain_cohere import ChatCohere
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

from diffusers import StableDiffusionPipeline

# ========= API KEY =========
COHERE_API_KEY = "bc9dj6eNiAgkZ614WHn81q3yGQeeBdmjh7jUEd6N"  # <-- put your key here
os.environ["COHERE_API_KEY"] = COHERE_API_KEY
co = cohere.Client(COHERE_API_KEY)

# ========= Stable Diffusion (text→image) =========
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE = torch.float16 if DEVICE == "cuda" else torch.float32
sd_pipe = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4", torch_dtype=DTYPE
).to(DEVICE)

# ========= Global KB (PDFs + Images OCR text) =========
embeddings = CohereEmbeddings(model="embed-english-v3.0")
kb = None                                   # global Chroma DB
persist_dir = "kb_db"                        # folder for persistence (optional)
memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True
)

# =========================
# 3) Helper functions
# =========================
def _ensure_db():
    """Create empty Chroma if not exists."""
    global kb
    if kb is None:
        kb = Chroma.from_texts([], embeddings, persist_directory=persist_dir)

def _add_text_to_kb(text: str) -> str:
    """Chunk text and add to vector store."""
    if not text or not text.strip():
        return "⚠️ No text to add."
    _ensure_db()
    splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=120)
    chunks = splitter.split_text(text)
    if not chunks:
        return "⚠️ No chunks produced."
    kb.add_texts(chunks)
    try:
        kb.persist()
    except Exception:
        pass
    return f"✅ Added {len(chunks)} chunks to knowledge base."

# ---- PDF(s) → text → KB
def add_pdfs_to_kb(pdf_files):
    if not pdf_files:
        return "⚠️ Please upload at least one PDF."
    collected = []
    for f in pdf_files:
        loader = PyPDFLoader(f.name if hasattr(f, "name") else f)
        docs = loader.load()
        collected.extend([d.page_content for d in docs if d.page_content])
    text = "\n".join(collected)
    return _add_text_to_kb(text)

# ---- Image(s) OCR → text → KB
def add_images_to_kb(image_files):
    if not image_files:
        return "⚠️ Please upload at least one image."
    extracted = []
    for f in image_files:
        path = f.name if hasattr(f, "name") else f
        img = Image.open(path)
        extracted.append(pytesseract.image_to_string(img))
    text = "\n".join(extracted)
    return _add_text_to_kb(text)

# ---- RAG Chat (uses KB if available)
def rag_chat(message, history):
    _ensure_db()
    llm = ChatCohere(model="command-r", temperature=0)
    chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=kb.as_retriever(search_kwargs={"k": 4}),
        memory=memory,
    )
    resp = chain({"question": message})
    return resp["answer"]

# ---- Normal chatbot (no KB)
def normal_chat(message, history):
    out = co.generate(model="command-r-plus", prompt=message, max_tokens=256)
    return out.generations[0].text.strip()

# ---- Text → Image
def text_to_image(prompt):
    image = sd_pipe(prompt).images[0]
    return image

# ---- Text → Speech (returns filepath)
def text_to_speech(text):
    if not text or not text.strip():
        return None
    out_path = "tts_output.mp3"
    gTTS(text).save(out_path)
    return out_path

# ---- Speech → Text (mp3/wav)
def speech_to_text(audio_path):
    if not audio_path:
        return "⚠️ Upload an audio file."
    # convert to mono 16k wav
    sound = AudioSegment.from_file(audio_path)
    sound = sound.set_channels(1).set_frame_rate(16000)
    wav_path = "converted.wav"
    sound.export(wav_path, format="wav")
    r = sr.Recognizer()
    with sr.AudioFile(wav_path) as src:
        audio = r.record(src)
    try:
        return r.recognize_google(audio, language="en-US")
    except sr.UnknownValueError:
        return "❌ Could not understand audio."
    except sr.RequestError as e:
        return f"❌ STT request error: {e}"

# ---- Clear KB
def clear_kb():
    global kb
    kb = None
    try:
        import shutil
        shutil.rmtree(persist_dir, ignore_errors=True)
    except Exception:
        pass
    memory.clear()
    return "🧹 Cleared knowledge base and chat memory."

# =========================
# 4) Gradio UI
# =========================
with gr.Blocks(title="Multimodal Assistant") as demo:
    gr.Markdown("# 🧠 Multimodal AI Assistant\n**PDF + Image OCR + RAG Chat + Normal Chat + Text→Image + STT + TTS**")

    with gr.Row():
        clear_btn = gr.Button("Clear KB & Memory", variant="stop")
        kb_status = gr.Markdown("")
        clear_btn.click(fn=clear_kb, outputs=kb_status)

    with gr.Tab("📄 Add PDFs to Knowledge Base"):
        pdf_upl = gr.Files(label="Upload one or more PDFs", file_types=[".pdf"])
        pdf_status = gr.Textbox(label="Status", interactive=False)
        add_pdf_btn = gr.Button("Add to KB")
        add_pdf_btn.click(add_pdfs_to_kb, inputs=[pdf_upl], outputs=[pdf_status])

    with gr.Tab("🖼 Add Images (OCR) to Knowledge Base"):
        img_upl = gr.Files(label="Upload one or more images", file_types=[".png", ".jpg", ".jpeg"])
        img_status = gr.Textbox(label="Status", interactive=False)
        add_img_btn = gr.Button("Add to KB")
        add_img_btn.click(add_images_to_kb, inputs=[img_upl], outputs=[img_status])

    with gr.Tab("💬 Chat over Your KB (RAG)"):
        gr.Markdown("Ask questions grounded in your uploaded PDFs & images.")
        rag_chat_ui = gr.ChatInterface(
            fn=rag_chat,
            textbox=gr.Textbox(placeholder="Ask about your documents..."),
            title="RAG Chat",
        )

    with gr.Tab("🗣️ Normal Chatbot (No KB)"):
        norm_chat_ui = gr.ChatInterface(
            fn=normal_chat,
            textbox=gr.Textbox(placeholder="Chat freely..."),
            title="Normal Chatbot",
        )

    with gr.Tab("🎨 Text → Image"):
        prompt = gr.Textbox(label="Prompt", placeholder="e.g., An apple wearing sunglasses, studio lighting")
        gen_img = gr.Image(label="Generated Image")
        gen_btn = gr.Button("Generate")
        gen_btn.click(text_to_image, inputs=prompt, outputs=gen_img)

    with gr.Tab("🎤 Speech → Text"):
        stt_audio = gr.Audio(source="upload", type="filepath", label="Upload MP3/WAV")
        stt_text = gr.Textbox(label="Transcription")
        stt_audio.change(speech_to_text, inputs=stt_audio, outputs=stt_text)

    with gr.Tab("🔊 Text → Speech"):
        tts_in = gr.Textbox(label="Text")
        tts_out = gr.Audio(type="filepath", label="Speech")
        tts_btn = gr.Button("Speak")
        tts_btn.click(text_to_speech, inputs=tts_in, outputs=tts_out)

demo.launch(share=True)


In [ ]:
!pip install gradio torch transformers diffusers sentence-transformers pypdf gTTS SpeechRecognition pydub --quiet


In [ ]:
import gradio as gr
import torch
from transformers import pipeline
from diffusers import StableDiffusionPipeline
from gtts import gTTS
import speech_recognition as sr
from pypdf import PdfReader


In [ ]:
# Text-to-Image
sd_pipe = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16
)
sd_pipe = sd_pipe.to("cuda" if torch.cuda.is_available() else "cpu")

# Image-to-Text
img_captioner = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")

# Chatbot
chatbot_pipe = pipeline("text-generation", model="gpt2")

# Speech recognizer
recognizer = sr.Recognizer()


In [ ]:
# --- Text to Image ---
def text_to_image(prompt):
    return sd_pipe(prompt).images[0]

# --- Image to Text ---
def image_to_text(image):
    return img_captioner(image)[0]["generated_text"]

# --- Chatbot ---
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load DialoGPT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

chatbot_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Function to maintain conversation
llm = Cohere(temperature=0, max_tokens=512)
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, memory=memory, verbose=False)
def chat_fn(user_input, history):
    if user_input.lower() == "quit":
        memory.clear()
        return history + [[user_input, "Conversation ended. Memory cleared."]]

    response = conversation.predict(input=user_input)
    history = history + [[user_input, response]]
    return history

def pdf_chat(pdf_file, question):
    reader = PdfReader(pdf_file.name)
    text = " ".join([page.extract_text() for page in reader.pages])
    qa_pipe = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
    answer = qa_pipe(question=question, context=text)
    return answer["answer"]

# --- Speech to Text ---
def speech_to_text(audio):
    with sr.AudioFile(audio) as source:
        audio_data = recognizer.record(source)
        return recognizer.recognize_google(audio_data)

# --- Text to Speech ---
def text_to_speech(text):
    tts = gTTS(text)
    tts.save("output.mp3")
    return "output.mp3"


In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# 🧠 Multimodal AI Assistant")
    with gr.Tab("💬Chatbot"):
      chatbot = gr.Chatbot()
      msg = gr.Textbox(placeholder="Type a message...")
      clear = gr.Button("Clear Chat")
      def user_message(message, history):
        return "", chat_fn(message, history)
      msg.submit(user_message, [msg, chatbot], [msg, chatbot])
      clear.click(lambda: (memory.clear(), []), None, chatbot, queue=False)
    with gr.Tab("📄 PDF Q&A"):
      pdf_input = gr.File(type="filepath", file_types=[".pdf"])  # ← changed here
      question = gr.Textbox(label="Ask a question")
      pdf_output = gr.Textbox()
      gr.Button("Get Answer").click(pdf_chat, [pdf_input, question], pdf_output)


    with gr.Tab("🖼️ Text → Image"):
        text_input = gr.Textbox(label="Enter prompt")
        img_output = gr.Image()
        gr.Button("Generate").click(text_to_image, text_input, img_output)

    with gr.Tab("🖼️ Image → Text"):
        img_input = gr.Image(type="filepath")
        caption_output = gr.Textbox()
        gr.Button("Describe").click(image_to_text, img_input, caption_output)

    with gr.Tab("🎤 Speech → Text"):
        audio_input = gr.Audio(sources=["microphone", "upload"], type="filepath")
        stt_output = gr.Textbox()
        gr.Button("Transcribe").click(speech_to_text, audio_input, stt_output)

    with gr.Tab("🗣️ Text → Speech"):
        tts_input = gr.Textbox()
        tts_output = gr.Audio()
        gr.Button("Speak").click(text_to_speech, tts_input, tts_output)

# Launch with a public link
demo.launch(share=True)


In [ ]:
!pip install gradio torch transformers diffusers sentence-transformers pypdf gTTS SpeechRecognition pydub --quiet


In [ ]:
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from diffusers import StableDiffusionPipeline
from gtts import gTTS
import speech_recognition as sr
from pypdf import PdfReader


In [ ]:
# Text-to-Image
sd_pipe = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16
)
sd_pipe = sd_pipe.to("cuda" if torch.cuda.is_available() else "cpu")

# Image-to-Text
img_captioner = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")

# Chatbot (DialoGPT)
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
chatbot_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Speech recognizer
recognizer = sr.Recognizer()


In [ ]:
# --- Text to Image ---
def text_to_image(prompt):
    return sd_pipe(prompt).images[0]

# --- Image to Text ---
def image_to_text(image):
    return img_captioner(image)[0]["generated_text"]

# --- Chatbot with messages dict format ---
def chatbot_fn(message, history):
    history = history or []

    # Build prompt from previous conversation
    prompt = ""
    for msg in history:
        if msg["role"] == "user":
            prompt += f"User: {msg['content']}\n"
        else:
            prompt += f"Bot: {msg['content']}\n"
    prompt += f"User: {message}\nBot:"

    # Generate response
    response = chatbot_pipe(
        prompt,
        max_length=len(prompt.split()) + 50,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True
    )[0]["generated_text"]

    bot_reply = response.split("Bot:")[-1].strip()

    # Update history in dict format
    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": bot_reply})

    return history, history

# --- PDF Q&A ---
def pdf_chat(pdf_file, question):
    reader = PdfReader(pdf_file)
    text = " ".join([page.extract_text() for page in reader.pages])
    qa_pipe = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
    answer = qa_pipe(question=question, context=text)
    return answer["answer"]

# --- Speech to Text ---
def speech_to_text(audio):
    with sr.AudioFile(audio) as source:
        audio_data = recognizer.record(source)
        return recognizer.recognize_google(audio_data)

# --- Text to Speech ---
def text_to_speech(text):
    tts = gTTS(text)
    tts.save("output.mp3")
    return "output.mp3"


In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# 🧠 Multimodal AI Assistant")

    # --- Chatbot ---
    with gr.Tab("💬 Chatbot"):
        chatbot = gr.Chatbot(type="messages")  # <-- messages format
        msg = gr.Textbox()
        clear = gr.Button("Clear")
        msg.submit(chatbot_fn, [msg, chatbot], [chatbot, chatbot])
        clear.click(lambda: [], None, chatbot, queue=False)

    # --- PDF Q&A ---
    with gr.Tab("📄 PDF Q&A"):
        pdf_input = gr.File(type="filepath", file_types=[".pdf"])
        question = gr.Textbox(label="Ask a question")
        pdf_output = gr.Textbox()
        gr.Button("Get Answer").click(pdf_chat, [pdf_input, question], pdf_output)

    # --- Text to Image ---
    with gr.Tab("🖼️ Text → Image"):
        text_input = gr.Textbox(label="Enter prompt")
        img_output = gr.Image()
        gr.Button("Generate").click(text_to_image, text_input, img_output)

    # --- Image to Text ---
    with gr.Tab("🖼️ Image → Text"):
        img_input = gr.Image(type="filepath")
        caption_output = gr.Textbox()
        gr.Button("Describe").click(image_to_text, img_input, caption_output)

    # --- Speech to Text ---
    with gr.Tab("🎤 Speech → Text"):
        audio_input = gr.Audio(sources=["microphone", "upload"], type="filepath")
        stt_output = gr.Textbox()
        gr.Button("Transcribe").click(speech_to_text, audio_input, stt_output)

    # --- Text to Speech ---
    with gr.Tab("🗣️ Text → Speech"):
        tts_input = gr.Textbox()
        tts_output = gr.Audio()
        gr.Button("Speak").click(text_to_speech, tts_input, tts_output)

# Launch with temporary public link in Colab
demo.launch(share=True, debug=True)


In [ ]:
!pip install streamlit pyngrok torch transformers diffusers sentence-transformers pypdf gTTS SpeechRecognition pydub --quiet


In [ ]:
pip install langchain-community


In [ ]:
import gradio as gr
from transformers import pipeline
import torch
import os
import PyPDF2
from PIL import Image
import pytesseract
import speech_recognition as sr
from gtts import gTTS
import tempfile

# ========== Pipelines ==========
chatbot_pipe = pipeline("text-generation", model="gpt2")
image_to_text_pipe = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")
text_to_image_pipe = pipeline("text-to-image", model="stabilityai/stable-diffusion-2", torch_dtype=torch.float16)

# ========== Functions ==========
def chatbot_response(history, message):
    response = chatbot_pipe(message, max_new_tokens=100, do_sample=True, temperature=0.7)[0]["generated_text"]
    history.append((message, response))
    return history, ""

def tts_response(text):
    tts = gTTS(text)
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as tmp:
        tts.save(tmp.name)
        return tmp.name

def stt_response(audio):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio) as source:
        audio_data = recognizer.record(source)
        try:
            return recognizer.recognize_google(audio_data)
        except sr.UnknownValueError:
            return "Could not understand audio."
        except sr.RequestError:
            return "API unavailable."

def pdf_qa(pdf, question):
    reader = PyPDF2.PdfReader(pdf.name)
    text = " ".join([page.extract_text() for page in reader.pages if page.extract_text()])
    qa_pipe = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
    answer = qa_pipe({"context": text, "question": question})
    return answer["answer"]

def image_to_text(img):
    return image_to_text_pipe(img)[0]['generated_text']

def text_to_image(prompt):
    image = text_to_image_pipe(prompt).images[0]
    out_path = tempfile.NamedTemporaryFile(delete=False, suffix=".png").name
    image.save(out_path)
    return out_path

# ========== UI ==========
with gr.Blocks() as demo:
    gr.Markdown("# 🚀 Multi-Feature AI App (Colab)")

    with gr.Tab("💬 Chatbot"):
        chatbot = gr.Chatbot()
        msg = gr.Textbox(label="Message")
        clear = gr.Button("Clear")
        msg.submit(chatbot_response, [chatbot, msg], [chatbot, msg])
        clear.click(lambda: None, None, chatbot, queue=False)

    with gr.Tab("🗣️ Text-to-Speech"):
        tts_in = gr.Textbox(label="Enter text")
        tts_out = gr.Audio()
        tts_in.submit(tts_response, tts_in, tts_out)

    with gr.Tab("🎤 Speech-to-Text"):
        stt_in = gr.Audio(sources=["microphone"], type="filepath")
        stt_out = gr.Textbox()
        stt_in.change(stt_response, stt_in, stt_out)

    with gr.Tab("📄 PDF Q&A"):
        pdf_file = gr.File(label="Upload PDF", file_types=[".pdf"])
        question = gr.Textbox(label="Ask a question from the PDF")
        pdf_out = gr.Textbox()
        question.submit(pdf_qa, [pdf_file, question], pdf_out)

    with gr.Tab("🖼️ Image-to-Text"):
        img_in = gr.Image(type="pil")
        img_out = gr.Textbox()
        img_in.upload(image_to_text, img_in, img_out)

    with gr.Tab("🎨 Text-to-Image"):
        txt2img_in = gr.Textbox(label="Enter prompt")
        txt2img_out = gr.Image()
        txt2img_in.submit(text_to_image, txt2img_in, txt2img_out)

demo.launch(share=True)


In [ ]:
# --- Chatbot ---
def chatbot_fn(user_input, history):
    if user_input.lower() == "quit":
        memory.clear()
        history.append([user_input, "Conversation ended. Memory cleared."])
        return history, history

    reply = conversation.predict(input=user_input)
    history = history + [[user_input, reply]]
    return history, history

# --- Text to Image ---
def text_to_image(prompt):
    return sd_pipe(prompt).images[0]

# --- Image to Text ---
def image_to_text(image):
    return img_captioner(image)[0]["generated_text"]

# --- PDF Q&A ---
def pdf_chat(pdf_file, question):
    reader = PdfReader(pdf_file)
    text = " ".join([page.extract_text() for page in reader.pages])
    qa_pipe = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
    answer = qa_pipe(question=question, context=text)
    return answer["answer"]

# --- Speech to Text ---
def speech_to_text(audio):
    with sr.AudioFile(audio) as source:
        audio_data = recognizer.record(source)
        return recognizer.recognize_google(audio_data)

# --- Text to Speech ---
def text_to_speech(text):
    tts = gTTS(text)
    tts.save("output.mp3")
    return "output.mp3"
